In [1]:
using Revise
using Pkg

Pkg.add("GPUArrays")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


┌ Warning: The active manifest file at `/home/kang/DRC_Project/DRC_Trajectron/Manifest.toml` has an old format that is being maintained.
│ To update to the new format, which is supported by Julia versions ≥ 1.6.2, run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
│ To then record the julia version re-resolve with `Pkg.resolve()` and if there are resolve conflicts consider `Pkg.update()`.
└ @ Pkg.Types /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/manifest.jl:287
  No Changes

 to `~/DRC_Project/DRC_Trajectron/Project.toml`


  No Changes to `~/DRC_Project/DRC_Trajectron/Manifest.toml`


┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /home/kang/DRC_Project/DRC_Trajectron/Manifest.toml:0


In [2]:
ENV["PYTHON"] = Sys.which("python")
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
Pkg.build("PyCall")
using FileIO
using JLD2
using RiskSensitiveSAC

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/8c86e48c0db1564a1d49548d3515ced5d604c408/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


┌ Info: Number of Julia Thread(s): 1
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:39
┌ Info: CUDA Device: NVIDIA GeForce RTX 3070 Laptop GPU
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:40


┌ Info: Python executable used by PyCall: /home/kang/anaconda3/envs/trajectron++/bin/python
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:41


In [3]:
Pkg.add("CSV")
Pkg.add("DataFrames")

using CSV
using DataFrames

   Resolving package versions...


  No Changes to `~/DRC_Project/DRC_Trajectron/Project.toml`
  No Changes to `~/DRC_Project/DRC_Trajectron/Manifest.toml`


   Resolving package versions...


  No Changes to `~/DRC_Project/DRC_Trajectron/Project.toml`
  No Changes to `~/DRC_Project/DRC_Trajectron/Manifest.toml`


In [4]:
include("$(@__DIR__)/../scripts/default_params/params_sensitivity_gaussian.jl");

σ_risk = 1.5;   

include("$(@__DIR__)/../scripts/parameter_setup.jl");

collision = [];
position_cost = [];
control_cost = [];
minimum_dist_list = [];

# Experiments

In [5]:
for experiment_iter in 1:100
    print("Experiment $experiment_iter: ")

    scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed = 
    controller_setup(scene_param,
                     predictor_param,
                     prediction_device=prediction_device,
                     cost_param=cost_param,
                     cnt_param=cnt_param,
                     dtc=dtc,
                     dto=dto,
                     ado_pos_init_dict=ado_pos_init_dict,
                     ado_vel_dict=ado_vel_dict,
                     ego_pos_init_vec=ego_pos_init_vec,
                     ego_pos_goal_vec=ego_pos_goal_vec,
                     target_speed=target_speed,
                     sim_horizon=sim_horizon,
                     verbose=false);

    result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                        target_speed, measurement_schedule, target_trajectory,
                        pos_error_replan, nominal_control=false);

    minimum_dist = minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                                for w in result.w_history])
    
    
    print("Total Collision: $(result.total_col), ")
    print("Total Position Cost: $(result.total_pos_cost), ")
    print("Total Control Cost: $(result.total_cnt_cost), ")
    print("Minimum Distance: $minimum_dist, ")

    push!(collision, result.total_col)
    push!(position_cost, result.total_pos_cost)
    push!(control_cost, result.total_cnt_cost)
    push!(minimum_dist_list, minimum_dist)

    # make_gif(result, dtplot=0.1, fps=5, xlim=(-6., 6.), ylim=(-6., 6.), figsize=(600, 400), 
    #             legendfontsize=7, legend=:topright, markersize=5., filename="rssac_data_trajectron_$(experiment_iter).gif",
    #             show_nominal_trajectory=false)


    if experiment_iter % 50 == 0
        CSV.write("trajectron_rssac_$(experiment_iter).csv", DataFrame(collision=collision, position_cost=position_cost, control_cost=control_cost, minimum_dist=minimum_dist_list))
    end
end

Experiment 1: 

┌ Warning: Time 0.00 [sec]: SAC computation took 49.42 [sec], which exceeds the maximum computation time allowed.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/rs_sac_controller.jl:733


┌ Warning: Time 0.00 [sec]: This evaluation iteration took 0.242 [sec], which exceeds dtc.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/evaluation.jl:300


┌ Warning: Time 0.10 [sec]: This evaluation iteration took 0.099 [sec], which exceeds dtc.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/evaluation.jl:300


┌ Warning: Time 0.20 [sec]: SAC computation took 0.101 [sec], which exceeds the maximum computation time allowed.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/rs_sac_controller.jl:733
┌ Warning: Time 0.20 [sec]: This evaluation iteration took 0.101 [sec], which exceeds dtc.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/evaluation.jl:300


...

┌ Warning: Time 4.70 [sec]: This evaluation iteration took 0.089 [sec], which exceeds dtc.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/evaluation.jl:300


┌ Warning: Time 4.80 [sec]: This evaluation iteration took 0.099 [sec], which exceeds dtc.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/evaluation.jl:300


┌ Warning: Time 4.90 [sec]: SAC computation took 0.11 [sec], which exceeds the maximum computation time allowed.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/rs_sac_controller.jl:733
┌ Warning: Time 4.90 [sec]: This evaluation iteration took 0.111 [sec], which exceeds dtc.
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/evaluation.jl:300


Excessive output truncated after 524479 bytes.

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] setindex!
        @ ~/.julia/packages/GPUArrays/Zecv7/src/host/indexing.jl:109 [inlined]
      [2] repeat_outer(arr::CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}, dims::NTuple{4, Int64})
        @ Base._RepeatInnerOuter ./abstractarraymath.jl:497
      [3] repeat_inner_outer
        @ ./abstractarraymath.jl:459 [inlined]
      [4] repeat(arr::CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}; inner::Nothing, outer::NTuple{4, Int64})
        @ Base._RepeatInnerOuter ./abstractarraymath.jl:401
      [5] #repeat#239
        @ ./abstractarraymath.jl:393 [inlined]
      [6] get_perturbed_schedule(w_init::WorldState, sim_result::SimulationResult, u_perturb_arrays::Vector{Vector{Vector{Float64}}}, target_trajectory::OrderedCollections.OrderedDict{RobotOS.Time, Vector{Float64}}, prediction_dict::Dict{String, Array{Float64, 3}}, sim_param::SimulationParameter, cnt_param::ControlParame

In [6]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-6., 6.), ylim=(-6., 6.), figsize=(600, 400), 
         legendfontsize=7, legend=:topright, markersize=5., filename="1_synthetic_gaussian.gif",
         show_nominal_trajectory=false)